<a href="https://colab.research.google.com/github/fambargh/projectmastery/blob/master/OD_detection_test_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#clone the repo executed in 7.937 s
!git clone https://github.com/kbardool/keras-frcnn.git

Cloning into 'keras-frcnn'...
remote: Enumerating objects: 589, done.
remote: Total 589 (delta 0), reused 0 (delta 0), pack-reused 589
Receiving objects: 100% (589/589), 173.38 KiB | 730.00 KiB/s, done.
Resolving deltas: 100% (402/402), done.


In [3]:
#generate the path for work
cd keras-frcnn

/content/keras-frcnn


In [4]:
!ls

keras_frcnn  measure_map.py  requirements.txt  thesis_128.zip
LICENSE      README.md	     test_frcnn.py     train_frcnn.py


In [5]:
from zipfile import ZipFile
filename = "thesis_128.zip"

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [6]:
!ls

keras_frcnn  measure_map.py  requirements.txt  thesis_128      train_frcnn.py
LICENSE      README.md	     test_frcnn.py     thesis_128.zip


In [0]:
from google.colab import files
files.upload()

In [8]:
!pip install -r requirements.txt


     |████████████████████████████████| 204kB 8.7MB/s 
  Created wheel for Keras: filename=Keras-2.0.3-cp36-none-any.whl size=232964 sha256=d05c6960f9270f692f67c270dceaad8e5be4c7fba031684f3948f37f252535ea
  Stored in directory: /root/.cache/pip/wheels/a6/fb/de/faea9e49d563a35f198c6dede7f9260074b5beb8f9bffaaaa1
Successfully built Keras
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.3 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [9]:
cd /content/keras-frcnn


/content/keras-frcnn


In [10]:
!pwd


/content/keras-frcnn


In [11]:
cd /content/keras-frcnn


/content/keras-frcnn


In [0]:

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches

In [13]:
train = pd.read_csv('/content/keras-frcnn/thesis_128/annotate_128_train.csv')
train.head()

,image_names,cell_type,xmin,xmax,ymin,ymax
0,01_g.png,od,90,111,47,73
1,01_h.png,od,86,107,49,77
2,02_g.png,od,20,42,48,76
3,02_h.png,od,27,48,46,75
4,03_g.png,od,83,107,48,80


In [0]:
data = pd.DataFrame()
data['format'] = train['image_names']

# as the images are in image_128_train folder, add image_128_train before the image name
for i in range(data.shape[0]):
    data['format'][i] = '/content/keras-frcnn/thesis_128/image_128_train/' + data['format'][i]

# add xmin, ymin, xmax, ymax and class as per the format required
for i in range(data.shape[0]):
    data['format'][i] = data['format'][i] + ',' + str(train['xmin'][i]) + ',' + str(train['ymin'][i]) + ',' + str(train['xmax'][i]) + ',' + str(train['ymax'][i]) + ',' + train['cell_type'][i]

data.to_csv('annotate.txt', header=None, index=None, sep=' ')

In [16]:
cd /content/keras-frcnn

/content/keras-frcnn


In [17]:
#training the mode executed in 9091.672s
!python train_frcnn.py -o simple -p 'annotate.txt'

Using TensorFlow backend.
Parsing annotation files
Training images per class:
{'bg': 0, 'od': 617}
Num classes (including bg) = 2
Config has been written to config.pickle, and can be loaded when testing to ensure correct results
Num train samples 507
Num val samples 110







Instructions for updating:
keep_dims is deprecated, use keepdims instead
loading weights from resnet50_weights_tf_dim_ordering_tf_kernels.h5
Could not load pretrained model weights. Weights can be found in the keras application folder 		https://github.com/fchollet/keras/tree/master/keras/applications


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Starting training
Epoch 1/500

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




2019-12-20 13:49:56.882055: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports

In [19]:
from google.colab import files
files.upload()

Saving test_frcnn1 - Copy (4).py to test_frcnn1 - Copy (4).py


{'test_frcnn1 - Copy (4).py': b'from __future__ import division\nimport os\nimport cv2\nimport numpy as np\nimport sys\nimport pickle\nfrom optparse import OptionParser\nimport time\nfrom keras_frcnn import config\nfrom keras import backend as K\nfrom keras.layers import Input\nfrom keras.models import Model\nfrom keras_frcnn import roi_helpers\nimport csv\n\nsys.setrecursionlimit(40000)\n\nparser = OptionParser()\n\nparser.add_option("-p", "--path", dest="test_path", help="Path to test data.")\nparser.add_option("-n", "--num_rois", type="int", dest="num_rois",\n\t\t\t\thelp="Number of ROIs per iteration. Higher means more memory use.", default=32)\nparser.add_option("--config_filename", dest="config_filename", help=\n\t\t\t\t"Location to read the metadata related to the training (generated when training).",\n\t\t\t\tdefault="config.pickle")\nparser.add_option("--network", dest="network", help="Base network to use. Supports vgg or resnet50.", default=\'resnet50\')\n\n(options, args) = 

In [20]:
from zipfile import ZipFile
filename = "Copy of thesis_500.zip"

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [37]:
#test the model executed in 148.538s
!python test_frcnn1.py -p /content/keras-frcnn/image_500_test

Using TensorFlow backend.
{0: 'od', 1: 'bg'}







Instructions for updating:
keep_dims is deprecated, use keepdims instead
Loading weights from ./model_frcnn.hdf5



2019-12-20 17:20:00.179867: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2019-12-20 17:20:00.185306: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000165000 Hz
2019-12-20 17:20:00.185533: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x767e540 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-20 17:20:00.185567: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-20 17:20:00.187383: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-20 17:20:00.299150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983

In [38]:
!zip -r /content/keras-frcnn/test500_1.zip /content/keras-frcnn/resault1

  adding: content/keras-frcnn/resault1/ (stored 0%)
  adding: content/keras-frcnn/resault1/Im0590_g_ORIGA.png (deflated 2%)
  adding: content/keras-frcnn/resault1/drishtiGS_019.png (deflated 0%)
  adding: content/keras-frcnn/resault1/drishtiGS_011.png (deflated 0%)
  adding: content/keras-frcnn/resault1/Im0223_ORIGA.jpg (deflated 5%)
  adding: content/keras-frcnn/resault1/Im0265_ORIGA.jpg (deflated 5%)
  adding: content/keras-frcnn/resault1/Im0246_ORIGA.jpg (deflated 5%)
  adding: content/keras-frcnn/resault1/Im0324_ORIGA.jpg (deflated 5%)
  adding: content/keras-frcnn/resault1/drishtiGS_005.png (deflated 0%)
  adding: content/keras-frcnn/resault1/Im0301_ORIGA.jpg (deflated 5%)
  adding: content/keras-frcnn/resault1/Im0163_ORIGA.jpg (deflated 5%)
  adding: content/keras-frcnn/resault1/10_r.png (stored 0%)
  adding: content/keras-frcnn/resault1/Im0316_ORIGA.jpg (deflated 5%)
  adding: content/keras-frcnn/resault1/Im0551_g_ORIGA.png (deflated 3%)
  adding: content/keras-frcnn/resault1/Im

In [39]:
!zip -r /content/keras-frcnn/test500_2.zip /content/keras-frcnn/resault2

  adding: content/keras-frcnn/resault2/ (stored 0%)
  adding: content/keras-frcnn/resault2/Im0590_g_ORIGA.png (deflated 2%)
  adding: content/keras-frcnn/resault2/drishtiGS_019.png (deflated 0%)
  adding: content/keras-frcnn/resault2/drishtiGS_011.png (stored 0%)
  adding: content/keras-frcnn/resault2/Im0223_ORIGA.jpg (deflated 10%)
  adding: content/keras-frcnn/resault2/Im0265_ORIGA.jpg (deflated 10%)
  adding: content/keras-frcnn/resault2/Im0246_ORIGA.jpg (deflated 11%)
  adding: content/keras-frcnn/resault2/Im0324_ORIGA.jpg (deflated 11%)
  adding: content/keras-frcnn/resault2/drishtiGS_005.png (deflated 1%)
  adding: content/keras-frcnn/resault2/Im0301_ORIGA.jpg (deflated 11%)
  adding: content/keras-frcnn/resault2/Im0163_ORIGA.jpg (deflated 11%)
  adding: content/keras-frcnn/resault2/10_r.png (stored 0%)
  adding: content/keras-frcnn/resault2/Im0316_ORIGA.jpg (deflated 12%)
  adding: content/keras-frcnn/resault2/Im0551_g_ORIGA.png (deflated 2%)
  adding: content/keras-frcnn/resaul